# Caribbean Low Urban Historic Landscapes
This is an indicator for the 2023 Blueprint.

It uses data representing the National Register of Historic Places. There is a national database for this from the park service, but the data has not been maintained and the polygons have huge slivers that are clearly errors. We dont use the polygon data from the national dataset. We do use the point data from the national dataset, in the contiguous SE only use it in places where we don't have better data. In the Virgin Islands and Puerto Rico, I am testing using it also.

We have territory level data dat representing the National Register of Historic Places for PR and are hoping to get more data for VI. These data tend to be more regularly maintained. 

We are also exploring using historic polygons from the OpenStreetMap.

In [65]:
import time
start = time.time()

In [66]:
import os
import arcpy

In [67]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
NSourceWorkspace = r"F:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlaces\NRIS_CR_Standards_Public.gdb"
#SSourceWorkspace = r"F:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData"
OutWorkspace = r"D:\SE_Blueprint_2023\4_Indicators\CarLowUrbanHistoric\CaribbeanLowU.gdb"

In [68]:
# define final outputs
Out = r"D:\SE_Blueprint_2023\4_Indicators\CarLowUrbanHistoric\CaribbeanLowUrbanHistoricLandscapes.tif"

In [69]:
# define sub-indicator outputs to help with user support

In [70]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [71]:
# define additional inputs

# historic sites from the PR Historic Preservation Office
prPly= r"F:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData\NRHP Puerto Rico Shape Files\NRHP_PR_polygonal_districts.shp"
prLn= r"F:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData\NRHP Puerto Rico Shape Files\NRHP_PR_lineal_districts.shp"
prPt=r"F:\GIS_DATA\Cultural_Resources\NationalRegisterOfHistoricPlacesStateData\NRHP Puerto Rico Shape Files\NRHP_PR_individual_properties.shp"

# historic districts from Nikita Beck
viHd1= r"F:\GIS_DATA\Cultural_Resources\USVI_Data\HistricDistricts\CharlotteAmalie_historic_boundary.shp"
viHd2= r"F:\GIS_DATA\Cultural_Resources\USVI_Data\HistricDistricts\Christiansted_historic_boundary.shp"
viHd3= r"F:\GIS_DATA\Cultural_Resources\USVI_Data\HistricDistricts\Frederiksted_historic_boundary.shp"


# historic sites from OpenStreetMap
osmH= r"F:\GIS_DATA\Cultural_Resources\OpenStreetMap20230314\Historic.gpkg\main.Historic"

# culturally important places from Greg Guannel
#ciVi= r"F:\GIS_DATA\Cultural_Resources\StCroix_CulturallyImportantPlaces\Culturally Important Places\culturally_important_places.shp"

# historic sites from the VI Historic Preservation Office
#viKMZ= r"F:\GIS_DATA\Cultural_Resources\USVI_Data\My Places.kmz"

# land use used to calculate urban
lulc= r"F:\GIS_DATA\LanduseLandcover\Landfire\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"



### Start Analysis

In [72]:
# Set the workspace where I want to save my output
arcpy.env.workspace = OutWorkspace

In [73]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\CarLowUrbanHistoric\CaribbeanLowU.gdb


### Import and buffer Point data



In [10]:
# Make a list of the all the point data to be used in this project 
#Pt = [prPt, ciVi, "MergeNational"]

In [11]:
# Merge all the point feature classes from the State Level National Register of Historic Places, using the list 
# created above as the input
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
#    arcpy.management.Merge(Pt,'MergeAllPoint', 'NO_SOURCE_INFO')

In [12]:
# buffer the points by 100 meters
arcpy.analysis.Buffer(prPt, "PtBuff", "100 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\PtBuff'>

### Import and buffer Polygon data 


In [13]:
# select only historic places with names
arcpy.analysis.Select(osmH, "osmHName", "name IS NOT NULL")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\osmHName'>

In [14]:
# convert all named OSM historic polygons to a geodatabase
with arcpy.EnvManager(extent=Rextent):
    #arcpy.management.FeatureToPolygon(osmH, "osmH", None, "ATTRIBUTES")
    #arcpy.conversion.FeatureClassToFeatureClass(osmH, "osmH", "OSMhistoric", '', 'osm_id "osm_id" true true false 2147483647 Text 0 0,First,#,main.historic,osm_id,0,2147483647;name "name" true true false 2147483647 Text 0 0,First,#,main.historic,name,0,2147483647;historic "historic" true true false 2147483647 Text 0 0,First,#,main.historic,historic,0,2147483647;other_tags "other_tags" true true false 2147483647 Text 0 0,First,#,main.historic,other_tags,0,2147483647', '')
    arcpy.management.FeatureToPolygon("osmHName", "osmHName2", None, "ATTRIBUTES", None)

In [15]:
# reproject OSM historic
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures("osmHName2", "osmHName3", '', None, None, None)

In [16]:
# Make a list of the polygon data
Ply = [prPly,viHd1, viHd2, viHd3,'osmHName3']

In [17]:
# Merge all the polygon feature classes from the State level National Register of Historic Places, using the list 
# created above as the input
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.Merge(Ply,'MergePoly', 'NO_SOURCE_INFO')

In [18]:
# Buffer the merged polygon layer by 30 meters. We did this because some of the polygons are so small that they 
# weren't represented when we convereted to 30 meter rasters. The buffer allows them to be represented in the raster.
arcpy.analysis.Buffer("MergePoly", "PolyBuff", "30 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\PolyBuff'>

## Import and buffer lines

In [19]:
# Buffer the line layer by 30 meters. 

# not sure if I should use 30 or 100 here, so I went with 30

arcpy.analysis.Buffer(prLn, "LnBuff", "30 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\LnBuff'>

## Make a raster of all historic places

Now we merge together the 3 different layers we made above: buffered national points for states we don't have data for, buffered state points, buffered state polygons

In [20]:
# Merge 3 processed inputs we made above
arcpy.management.Merge(["PtBuff","PolyBuff","LnBuff"], "MergeAllBuffered", "NO_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\MergeAllBuffered'>

In [21]:
# add and calculate a field that we can use to convert to a raster
arcpy.management.CalculateField("MergeAllBuffered", "raster", "1", "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\MergeAllBuffered'>

In [22]:
# convert all historic polygon to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("MergeAllBuffered", "raster", "MergeAllBufferedr", "CELL_CENTER", "NONE", Rextent)

### Bring in Landcover

These steps take the landcover, pull out all urban classes as 1 and leave everything else as 0, then uses focal statistics\mean to calculate the percent urban in a 270 meter radius (we used 270 instead of 250 since we are using 30 meter pixels).  All pixels that are < 50% urban in a 270 meter radius are considered low urban and are given a value of 1.  All pixels that are > 50% urban in a 279 meter radius are considered high urban and are give a value of 0. 

In [23]:
# In the contiguous US, we used NLCD and made a raster with the 4 urban classes as 1 and everything else is 0
# to match those methods, I"m making all 5 Landvire Developed classes 1 and everything else 0
# I'm now wondering why we made NLCD Developed, Open Space 1?
# Perhaps we should only use the higher developed classes?
with arcpy.EnvManager(extent=Rextent):
    out_raster = arcpy.sa.Con(lulc, 1, 0, "EVT_NAME IN ('Developed-High Intensity', 'Developed-Low Intensity', 'Developed-Medium Intensity', 'Developed-Open Space', 'Developed-Roads')"); out_raster.save("lulcUrb")

In [24]:
# calculate focal statistics on above raster. Cacluate mean in a circle neighborhood with a radius of 270 meters. This will give us the
# percent urban in that focal window (because of how we reclassified above)
# This ran before, and it runs now, but now I keep getting an error that I cannot save this to my workspace. Grr.
# I'm going to try to save as a .tif to see if that works
# nope still got the runtime error
# problem solved, my harddrive was running out of space
out_raster = arcpy.sa.FocalStatistics("lulcUrb", "Circle 270 MAP", "MEAN", "DATA", 90); out_raster.save("M270m")

In [25]:
# reclassify so that all pixels that are > 50% urban in a 279 meter radius are considered high urban and are give a value of 0
out_raster = arcpy.sa.Reclassify("M270m", "VALUE", "0 0.500000 1;0.500000 1 0", "DATA"); out_raster.save("M270m50T")

In [26]:
# Landfire is already in the same projection we use for the blueprint, so don't need to worry about reprojecting

In [27]:
# Mask out areas that are not historic by taking the percent urban raster times the historic areas raster
out_raster = arcpy.sa.Times("MergeAllBufferedr", "M270m50T"); out_raster.save("M270m50TH")

In [28]:
# We decided in the end to change the classification so that 0 = not a historic site, 1 = high urban historic site, 2 = low urban historic site
out_raster = arcpy.sa.Reclassify("M270m50TH", "VALUE", "0 1;1 2;NODATA 0" "DATA"); out_raster.save("prelim")

### need to figure out what extent is for 0/NoData
This is probably a round about way to do it, but I am just trying to get a draft out

In [29]:
# reclassify to get 0/NoData
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(lulc, 1, '', "EVT_NAME <> 'Fill-NoData'"); out_raster.save("lfEx")

In [30]:
# Mask out areas that are not covered by land cover data
out_raster = arcpy.sa.Times("lfEx", "prelim"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [31]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return '2 = Historic place with nearby low-urban buffer'
    elif value == 1:
        return '1 = Historic place with nearby high-urban buffer'
    elif value == 0:
        return '0 = Not identified as a historic place'    
"""

In [32]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarLowUrbanHistoric\\CaribbeanLowU.gdb\\indicator'>

In [74]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 93
	if Value == 1:
		return 161
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 64
	if Value == 1:
		return 136
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 55
	if Value == 1:
		return 127
	else:
		return 255
		
"""

In [75]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [76]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("indicator1")

In [77]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("indicator1", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [37]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

43.57563877105713
